# Simulation of 1D Shallow Water Equations

## Numerical Solution using Finite Element Method

In [1]:
import numpy as np

import inputs
import grid_generation
import basis
import initial_conditions
import evolve 
import integrator
import plots
import test

# running some test
test.test_lagrange_basis()
test.test_lagrange_basis_derivative()
test.test_integration()
# test.test_mass_matrix()

# creating mesh
nodes_coordinates_phys_space = grid_generation.generate_1d_mesh(inputs.x_initial,inputs.x_final,inputs.N_elements,inputs.p_basis_order)

element_number = np.arange(inputs.N_elements)
element_lengths = nodes_coordinates_phys_space[:,-1] - nodes_coordinates_phys_space[:,0]

# generating reference space information
gauss_weights, basis_values_at_gauss_quad, basis_values_x_derivative_at_gauss_quad, basis_values_at_nodes = basis.generate_reference_space(nodes_coordinates_phys_space,inputs.n_gauss_poins)

# generating initial conditions
h, u = initial_conditions.generate_initial_conditions(nodes_coordinates_phys_space)

# compute entropy : integral 0.5 * ( g * h**2 + h * u ) dx
entropy = evolve.compute_entropy(element_lengths, gauss_weights, basis_values_at_gauss_quad, h, u)

# wrinting initial conditions file
integrator.write_data_file(nodes_coordinates_phys_space,entropy,h,u,False,0)

# compute mass matrix M_ij = integral phi_i(x) phi_j(x) dx and return the inverse matrix of M_ij
mass_matrix_inverse = evolve.compute_mass_matrix_inverse(element_lengths, gauss_weights, basis_values_at_gauss_quad)

# time step
time_step = np.array(inputs.t_limit/inputs.n_steps) 

# evolving in time the PDE
for number_of_t_step in np.arange(inputs.n_steps):

    # If true using euler method
    if inputs.evolution_method==0:
        h, u = integrator.euler_method( h, u, time_step, element_number, basis_values_at_gauss_quad, basis_values_x_derivative_at_gauss_quad, gauss_weights, element_lengths, basis_values_at_nodes, mass_matrix_inverse)

    # If true using RK4 method
    if inputs.evolution_method==1:
        h, u = integrator.rk4_method( h, u, time_step, element_number, basis_values_at_gauss_quad, basis_values_x_derivative_at_gauss_quad, gauss_weights, element_lengths, basis_values_at_nodes, mass_matrix_inverse)

    # saving the data
    if (number_of_t_step+1) % inputs.plot_every_steps == 0:

        # compute entropy : integral 0.5 * ( g * h**2 + h * u ) dx
        entropy = evolve.compute_entropy(element_lengths, gauss_weights, basis_values_at_gauss_quad, h, u)

        # writing data
        integrator.write_data_file(nodes_coordinates_phys_space,entropy,h,u,False,number_of_t_step+1)

# plotting data
plots.plotting()

print(f'Done')

Generating mesh 
Physical domain: [0,10] meters
Number of elements: 8
Nodes per element: 2
Lagrange basis order: 1
Saving initial conditions
Writing step 0 | t = 0.0
Writing step 400 | t = 0.4
Writing step 800 | t = 0.8
Writing step 1200 | t = 1.2
Writing step 1600 | t = 1.6
Writing step 2000 | t = 2.0
Writing step 2400 | t = 2.4
Writing step 2800 | t = 2.8
Writing step 3200 | t = 3.2
Writing step 3600 | t = 3.6
Writing step 4000 | t = 4.0
Plotting data
Plotting step_0.h5
Plotting step_400.h5
Plotting step_800.h5
Plotting step_1200.h5
Plotting step_1600.h5
Plotting step_2000.h5
Plotting step_2400.h5
Plotting step_2800.h5
Plotting step_3200.h5
Plotting step_3600.h5
Plotting step_4000.h5
Done
